In [144]:
import re
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN
from pptx.oxml import parse_xml
from pptx.oxml.ns import nsdecls

In [145]:

def parse_md_file(md_file):
    """Parses a markdown file and extracts slide titles and bullet points without labels."""
    slides = []
    with open(md_file, "r", encoding="utf-8") as file:
        content = file.read()

    # Split slides using Markdown headers (e.g., # Slide 1: Title)
    slide_sections = re.split(r"^# (.*?)$", content, flags=re.MULTILINE)[1:]

    for i in range(0, len(slide_sections), 2):
        title = re.sub(r"^(Title Slide:|Slide \d+:|Conclusion Slide:|References Slide:|Appendix Slide:)\s*", "", slide_sections[i].strip(), flags=re.IGNORECASE)  # Remove labels
        bullets = [line.strip("- ").strip() for line in slide_sections[i + 1].split("\n") if line.startswith("-")]
        slides.append((title, bullets))

    return slides

def set_text_format(text_frame, font_name="Arial", font_size=24, color=(0, 0, 0), align=PP_ALIGN.LEFT):
    """Applies font formatting and alignment to a text frame."""
    for paragraph in text_frame.paragraphs:
        paragraph.alignment = align  # Set text alignment (JUSTIFY used in content slides)
        run = paragraph.runs[0] if paragraph.runs else paragraph.add_run()
        run.font.name = font_name
        run.font.size = Pt(font_size)
        run.font.color.rgb = RGBColor(*color)  # Set font color

def parse_hyperlinked_text(text):
    """Extracts (text, URL) pairs from Markdown-style [text](url) hyperlinks."""
    pattern = r"\[(.*?)\]\((.*?)\)"
    return re.findall(pattern, text)

def add_hyperlinked_text(paragraph, text, url):
    """Adds only the hyperlinked text to a paragraph in PowerPoint (without raw MD syntax)."""
    paragraph.clear()  # Remove any default text
    run = paragraph.add_run()
    run.text = text  # Display only the text, not the URL
    run.font.color.rgb = RGBColor(0, 0, 255)  # Blue color for hyperlink
    run.font.size = Pt(20)
    run.hyperlink.address = url  # Make it clickable

In [146]:
slides = parse_md_file(md_file="slides.md")

In [147]:
slides

[('Biodiversity Presentation', []),
 ('Introduction to Biodiversity',
  ['Biodiversity refers to the variety of life, including species, ecosystems, and genetic diversity.',
   'It maintains ecological balance and supports essential ecosystem services.',
   'Found in terrestrial, freshwater, and marine environments worldwide.']),
 ('Importance of Biodiversity',
  ['Supports ecosystem stability and resilience to environmental changes.',
   'Provides essential resources like food, medicine, and raw materials.',
   'Contributes to climate regulation, water purification, and soil fertility.']),
 ('Threats to Biodiversity',
  ['Habitat destruction from deforestation, urbanization, and agriculture.',
   'Climate change causing species extinction and ecosystem shifts.',
   'Pollution harming air, water, and soil quality, affecting biodiversity.']),
 ('Conclusion',
  ['Biodiversity is essential for ecosystem stability and human well-being.',
   'Conservation efforts are crucial to protect spec

In [148]:
from datetime import date
date.today()

datetime.date(2025, 2, 24)

In [149]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN
from datetime import date


def create_ppt_from_md(md_file, output_pptx):
    """Creates a PowerPoint file from a markdown file with justified bullet points and a distinct title slide."""
    slides_content = parse_md_file(md_file)
    prs = Presentation()

    slide_titles = []  # Store content slide titles for the outline
    appendix_titles = []
    
    for i, (title, bullet_points) in enumerate(slides_content):
        if i == 0 and not bullet_points:
            # Title Slide
            slide = prs.slides.add_slide(prs.slide_layouts[0])
            title_placeholder = slide.shapes.title
            subtitle_placeholder = slide.placeholders[1]

            title_placeholder.text = title
            subtitle_placeholder.text = ""

            # Add date at the bottom-right corner
            left, top = Inches(8.5), Inches(6.9)
            date_textbox = slide.shapes.add_textbox(left, top, Inches(1), Inches(0.5))
            date_text_frame = date_textbox.text_frame
            date_text_frame.text = date.today().strftime("%d-%m-%Y")

            # Apply font formatting
            set_text_format(title_placeholder.text_frame, font_name="Calibri", font_size=44, color=(0, 0, 128))
            set_text_format(subtitle_placeholder.text_frame, font_name="Calibri", font_size=24, color=(80, 80, 80))

        else:
            # Check if slide is an appendix slide
            if title.lower().startswith("appendix"):
                appendix_titles.append(title)
            else:
                slide_titles.append(title)
            # Content Slides
            slide = prs.slides.add_slide(prs.slide_layouts[1])
            title_placeholder = slide.shapes.title
            content_placeholder = slide.placeholders[1]

            title_placeholder.text = title
            content_placeholder.text = ""  # Clear placeholder text

            # Add bullet points with justified alignment
            for point in bullet_points:
                paragraph = content_placeholder.text_frame.add_paragraph()
                paragraph.text = f"{point}"
                paragraph.alignment = PP_ALIGN.JUSTIFY  # Justify text
                paragraph.space_after = Pt(8)
                
                hyperlinks = parse_hyperlinked_text(point)
                if hyperlinks:
                    for text, url in hyperlinks:
                        add_hyperlinked_text(paragraph, text, url)  # Add clickable hyperlink
                else:
                    paragraph.text = point  # Regular text

                # Apply font formatting
                run = paragraph.runs[0]
                run.font.name = "Arial"
                run.font.size = Pt(20)
                run.font.color.rgb = RGBColor(50, 50, 50)  # Dark Gray

            # Apply formatting to title
            set_text_format(title_placeholder.text_frame, font_name="Arial", font_size=32, color=(0, 0, 0))

            # Add slide number (bottom-right corner)
            left, top = Inches(8.5), Inches(6.9)
            slide_number = slide.shapes.add_textbox(left, top, Inches(1), Inches(0.5))
            slide_number.text_frame.text = f"{i}/{len(slides_content) - 1}"
            set_text_format(slide_number.text_frame, font_name="Arial", font_size=14, color=(100, 100, 100), align=PP_ALIGN.RIGHT)
    
    # Create an Outline Slide after the Title Slide
    outline_slide = prs.slides.add_slide(prs.slide_layouts[1])  # Title & Content Layout
    outline_slide.shapes.title.text = "Outline"
    outline_text_frame = outline_slide.placeholders[1].text_frame
    outline_text_frame.clear()

    for slide_title in slide_titles:
        paragraph = outline_text_frame.add_paragraph()
        paragraph.text = slide_title  # Add slide title as bullet point
        paragraph.font.size = Pt(20)
        paragraph.font.color.rgb = RGBColor(0, 0, 0)  # Black
    
    if appendix_titles:
        paragraph = outline_text_frame.add_paragraph()
        paragraph.text = "Appendices"  # Add slide title as bullet point
        paragraph.font.size = Pt(20)
        paragraph.font.color.rgb = RGBColor(0, 0, 0)  # Black
    
    # Move Outline Slide to position 2 (after the Title Slide)
    xml_slides = prs.slides._sldIdLst  # Access slide list
    xml_slides.insert(1, xml_slides[-1])  # Move last slide to index 1 (second position)
    
    prs.save(output_pptx)
    print(f"PowerPoint file '{output_pptx}' has been created successfully with justified bullet points!")


In [150]:

# Example usage
md_filename = "slides.md"  # Your Markdown file
pptx_filename = "biodiversity.pptx"  # Output PowerPoint file

create_ppt_from_md(md_filename, pptx_filename)


PowerPoint file 'biodiversity.pptx' has been created successfully with justified bullet points!
